<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/main/models/base_models_STHyMOUNTAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Entrenamiento de Modelos Baseline para Predicción Espaciotemporal de Precipitación Mensual STHyMOUNTAIN

Este notebook implementa modelos baseline para la predicción de precipitaciones usando datos espaciotemporales.

## 🔍 Implementación de Modelos Avanzados y Técnicas de Validación

Además de los modelos tabulares baseline, implementaremos:

1. **Optimización avanzada con Optuna** para los modelos tabulares XGBoost y LightGBM
2. **Validación robusta** mediante:
   - Hold-Out Validation (ya implementada)
   - Cross-Validation (k=5)
   - Bootstrapping (100 muestras)
3. **Modelos de Deep Learning** para capturar patrones espaciales y temporales:
   - Redes CNN para patrones espaciales
   - Redes ConvLSTM para patrones espaciotemporales

El objetivo es proporcionar una evaluación completa de diferentes enfoques de modelado para la predicción de precipitación en regiones montañosas.

In [22]:
# Configuración del entorno (compatible con Colab y local)
import os
import sys
from pathlib import Path

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')   
    # Si estamos en Colab, clonar el repositorio
    !git clone https://github.com/ninja-marduk/ml_precipitation_prediction.git
    %cd ml_precipitation_prediction
    # Instalar dependencias necesarias
    !pip install -r requirements.txt
    !pip install xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn
    BASE_PATH = '/content/drive/MyDrive/ml_precipitation_prediction'
else:
    # Si estamos en local, usar la ruta actual
    if '/models' in os.getcwd():
        BASE_PATH = Path('..')
    else:
        BASE_PATH = Path('.')

print(f"Entorno configurado. Usando ruta base: {BASE_PATH}")

# Si BASE_PATH viene como string, lo convertimos
BASE_PATH = Path(BASE_PATH)

# Ahora puedes concatenar correctamente
model_output_dir = BASE_PATH / 'models' / 'output'
model_output_dir.mkdir(parents=True, exist_ok=True)

print(f"Directorio para salida de modelos creado: {model_output_dir}")


Entorno configurado. Usando ruta base: ..
Directorio para salida de modelos creado: ../models/output


In [23]:
# 1. Importaciones necesarias
import numpy as np
import pandas as pd
import xarray as xr
import optuna
import pickle
import datetime
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Importaciones para barras de progreso y mejora de visualización
from tqdm.notebook import tqdm, trange
from IPython.display import display, HTML, clear_output
import time

# Configurar visualización más atractiva
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
# Importaciones adicionales para Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, save_model, load_model
from tensorflow.keras.layers import (Dense, Dropout, Conv2D, Conv3D, ConvLSTM2D, BatchNormalization, 
                                   MaxPooling2D, Flatten, Input, concatenate, Reshape, TimeDistributed)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

print("TensorFlow versión:", tf.__version__)

# Configurar GPU si está disponible
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print(f"GPU disponible: {physical_devices}")
    # Permitir crecimiento de memoria según sea necesario
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    print("No se detectó GPU. Usando CPU.")

In [ ]:
# 2. Cargar el dataset NetCDF
def load_dataset(file_path):
    """Carga un archivo NetCDF y lo convierte a pandas DataFrame"""
    try:
        # Cargar el archivo NetCDF con xarray
        print(f"Intentando cargar el archivo: {file_path}")
        ds = xr.open_dataset(file_path)
        print("Archivo cargado exitosamente con xarray")

        # Mostrar información del dataset cargado
        print("\nInformación del dataset:")
        print(ds.info())
        print("\nVariables disponibles:")
        for var_name in ds.data_vars:
            print(f"- {var_name}: {ds[var_name].shape}")

        # Convertir a DataFrame
        df = ds.to_dataframe().reset_index()
        return df, ds
    except Exception as e:
        print(f"Error al cargar el archivo NetCDF: {e}")
        return None, None

# Ruta al dataset
data_file = BASE_PATH / 'data' / 'output' / 'complete_dataset_with_features.nc'
print(f"Buscando archivo en: {data_file}")

# Cargar el dataset
df, ds_original = load_dataset(data_file)

# Verificar si se cargó correctamente
if df is not None:
    print(f"Dataset cargado con éxito. Dimensiones: {df.shape}")
    print("\nPrimeras filas del DataFrame:")
    display(df.head())
else:
    print("No se pudo cargar el dataset. Verificar la ruta y el formato del archivo.")

In [ ]:
# 3. Preparación de los datos
if df is not None:
    # Identificar la columna objetivo (precipitación)
    target_column = 'total_precipitation'  # Ajustar si tiene otro nombre en tu dataset

    # Ver si existe 'precip_target' o usar 'total_precipitation'
    if 'total_precipitation' in df.columns:
        target_column = 'total_precipitation'

    print(f"Columna objetivo identificada: {target_column}")

    # Separar variables predictoras y variable objetivo
    feature_cols = [col for col in df.columns if col != target_column and not pd.isna(df[col]).all()]

    # Eliminar columnas no numéricas para los modelos (como fechas o coordenadas si no se usan como features)
    non_feature_cols = ['time', 'spatial_ref']
    feature_cols = [col for col in feature_cols if col not in non_feature_cols]

    # Eliminar filas con valores NaN
    print(f"Filas antes de eliminar NaN: {df.shape[0]}")
    df_clean = df.dropna(subset=[target_column] + feature_cols)
    print(f"Filas después de eliminar NaN: {df_clean.shape[0]}")

    # Separar features y target
    X = df_clean[feature_cols]
    y = df_clean[target_column]

    print(f"\nFeatures seleccionadas ({len(feature_cols)}):\n{feature_cols}")
    print(f"\nVariable objetivo: {target_column}")

In [27]:
# 4. División del conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensiones del conjunto de entrenamiento: {X_train.shape}")
print(f"Dimensiones del conjunto de prueba: {X_test.shape}")

# 5. Estandarización de variables predictoras
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Guardar el scaler para uso futuro
with open(model_output_dir / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("Escalador guardado en models/output/scaler.pkl")

Dimensiones del conjunto de entrenamiento: (1735008, 12)
Dimensiones del conjunto de prueba: (433752, 12)
Escalador guardado en models/output/scaler.pkl


In [28]:
# 6. Funciones de evaluación y entrenamiento
def evaluar_modelo(y_true, y_pred):
    """Evalúa el rendimiento de un modelo usando múltiples métricas"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

def entrenar_y_evaluar_modelo(modelo, nombre, X_train, y_train, X_test, y_test):
    """Entrena un modelo y evalúa su rendimiento con visualización del progreso"""
    # Crear widget para mostrar información del proceso
    display(HTML(f'<div style="background-color:#f0f8ff; padding:10px; border-radius:5px;">' +
                 f'<h3>🔄 Entrenando modelo: {nombre}</h3>' +
                 f'<div id="status_{nombre}">Estado: Iniciando entrenamiento...</div>' +
                 f'</div>'))
    
    # Tiempo de inicio
    start_time = time.time()
    
    # Entrenar el modelo con seguimiento visual según el tipo
    if hasattr(modelo, 'fit_generator') or nombre in ['XGBoost', 'XGBoost_Optuna', 'LightGBM', 'LightGBM_Optuna']:
        # Para modelos que soportan entrenamiento por lotes como XGBoost, LightGBM
        print(f"Entrenando {nombre} con visualización de progreso...")
        if hasattr(modelo, 'n_estimators'):
            n_estimators = modelo.n_estimators
            for i in tqdm(range(n_estimators), desc=f"Entrenando {nombre}"):
                if i == 0:
                    # Primera iteración, ajuste inicial
                    if nombre.startswith('LightGBM'):
                        # LightGBM tiene parámetro verbose
                        temp_modelo = type(modelo)(n_estimators=1, **{k:v for k,v in modelo.get_params().items() 
                                                                 if k != 'n_estimators' and k != 'verbose'}, verbose=-1)
                    else:
                        temp_modelo = type(modelo)(n_estimators=1, **{k:v for k,v in modelo.get_params().items() 
                                                                if k != 'n_estimators'})
                    temp_modelo.fit(X_train, y_train)
                elif i == n_estimators - 1:
                    # Última iteración, ajuste completo
                    modelo.fit(X_train, y_train)
                
                # Actualizar progreso visual
                if i % max(1, n_estimators // 10) == 0:
                    clear_output(wait=True)
                    display(HTML(f'<div style="background-color:#f0f8ff; padding:10px; border-radius:5px;">' +
                                f'<h3>🔄 Entrenando modelo: {nombre}</h3>' +
                                f'<div id="status_{nombre}">Estado: Progreso {i+1}/{n_estimators} estimadores ({((i+1)/n_estimators*100):.1f}%)</div>' +
                                f'</div>'))
                    time.sleep(0.1)  # Pequeña pausa para actualización visual
        else:
            # Si no tiene n_estimators, entrenamiento directo
            modelo.fit(X_train, y_train)
    else:
        # Para modelos estándar como RandomForest
        modelo.fit(X_train, y_train)
    
    # Tiempo de entrenamiento
    training_time = time.time() - start_time
    
    # Visualizar tiempo de entrenamiento
    display(HTML(f'<div style="background-color:#e6ffe6; padding:10px; border-radius:5px;">' +
                f'<h3>✅ Entrenamiento completado: {nombre}</h3>' +
                f'<div>Tiempo de entrenamiento: {training_time:.2f} segundos</div>' +
                f'</div>'))
    
    print(f"Evaluando rendimiento de {nombre}...")
    predicciones = modelo.predict(X_test)
    rmse, mae, r2 = evaluar_modelo(y_test, predicciones)
    
    # Visualizar métricas con estilo
    display(HTML(f'<div style="background-color:#f5f5dc; padding:10px; border-radius:5px; margin-top:10px;">' +
                f'<h3>📊 Métricas para {nombre}</h3>' +
                f'<table style="width:100%; text-align:left;">' +
                f'<tr><th>Métrica</th><th>Valor</th></tr>' +
                f'<tr><td>RMSE</td><td>{rmse:.4f}</td></tr>' +
                f'<tr><td>MAE</td><td>{mae:.4f}</td></tr>' +
                f'<tr><td>R²</td><td>{r2:.4f}</td></tr>' +
                f'</table></div>'))
    
    return modelo, (rmse, mae, r2)

def guardar_modelo(modelo, nombre):
    """Guarda un modelo entrenado en disco"""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{nombre}_{timestamp}.pkl"
    with open(model_output_dir / filename, 'wb') as f:
        pickle.dump(modelo, f)
    
    # Visualizar confirmación de guardado
    display(HTML(f'<div style="background-color:#e6ffee; padding:10px; border-radius:5px; margin-top:10px;">' +
                f'<h3>💾 Modelo guardado</h3>' +
                f'<div>Modelo <b>{nombre}</b> guardado como: {filename}</div>' +
                f'</div>'))
    return filename

In [ ]:
# 7. Entrenamiento de modelos base sin optimización

# Inicializar diccionarios para almacenar resultados y modelos
resultados_base = {}  # Para almacenar métricas (RMSE, MAE, R2)
modelos_base = {}     # Para almacenar instancias de modelos
modelos_guardados = {} # Para almacenar nombres de archivos guardados

print("\n🔍 Entrenando modelos baseline sin optimización de hiperparámetros...")

# 1. Modelo RandomForest básico
print("\nEntrenando RandomForest base...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model, rf_metrics = entrenar_y_evaluar_modelo(
    rf_model, 'RandomForest', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['RandomForest'] = rf_metrics
modelos_base['RandomForest'] = rf_model
modelo_file = guardar_modelo(rf_model, 'RandomForest')
modelos_guardados['RandomForest'] = modelo_file

# Visualizar importancia de características para RandomForest
plt.figure(figsize=(12, 6))
feat_importances = rf_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='skyblue')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - RandomForest')
plt.tight_layout()
plt.savefig(model_output_dir / 'randomforest_feature_importance.png')
plt.show()

# 2. Modelo XGBoost básico
print("\nEntrenando XGBoost base...")
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model, xgb_metrics = entrenar_y_evaluar_modelo(
    xgb_model, 'XGBoost', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['XGBoost'] = xgb_metrics
modelos_base['XGBoost'] = xgb_model
modelo_file = guardar_modelo(xgb_model, 'XGBoost')
modelos_guardados['XGBoost'] = modelo_file

# Visualizar importancia de características para XGBoost
plt.figure(figsize=(12, 6))
feat_importances = xgb_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='coral')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - XGBoost')
plt.tight_layout()
plt.savefig(model_output_dir / 'xgboost_feature_importance.png')
plt.show()

# 3. Modelo LightGBM básico
print("\nEntrenando LightGBM base...")
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
lgbm_model, lgbm_metrics = entrenar_y_evaluar_modelo(
    lgbm_model, 'LightGBM', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['LightGBM'] = lgbm_metrics
modelos_base['LightGBM'] = lgbm_model
modelo_file = guardar_modelo(lgbm_model, 'LightGBM')
modelos_guardados['LightGBM'] = modelo_file

# Visualizar importancia de características para LightGBM
plt.figure(figsize=(12, 6))
feat_importances = lgbm_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='lightgreen')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - LightGBM')
plt.tight_layout()
plt.savefig(model_output_dir / 'lightgbm_feature_importance.png')
plt.show()

# Comparar resultados de modelos base
print("\n🔍 Comparación de modelos base sin optimización:")
temp_df = pd.DataFrame(resultados_base, index=['RMSE', 'MAE', 'R2']).T
print("\nOrdenados por RMSE (menor es mejor):")
display(temp_df.sort_values('RMSE'))

# Visualizar comparación de RMSE
plt.figure(figsize=(10, 5))
sns.barplot(x=temp_df.index, y=temp_df['RMSE'])
plt.title('Comparación de RMSE - Modelos Base')
plt.ylabel('RMSE (menor es mejor)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(model_output_dir / 'baseline_rmse_comparison.png')
plt.show()

In [ ]:
# Optimización adaptativa de memoria RAM para Optuna (compatible con Colab y Local)
import gc
import psutil
import os
import tempfile
import time
import optuna
import numpy as np
from sklearn.metrics import mean_squared_error
from IPython.display import HTML
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

def get_available_memory():
    """Detecta la memoria RAM disponible en el sistema actual (Colab o local)"""
    try:
        # Obtener memoria disponible usando psutil
        mem_info = psutil.virtual_memory()
        available_mb = mem_info.available / (1024 * 1024)  # Convertir a MB
        total_mb = mem_info.total / (1024 * 1024)  # Convertir a MB
        
        # Detectar si estamos en Colab
        in_colab = 'google.colab' in sys.modules
        if in_colab:
            # En Colab, limitar el uso a un porcentaje conservador
            print("Entorno detectado: Google Colab")
            max_usage_percent = 70  # Usar máximo 70% de la memoria disponible en Colab
        else:
            # En local, también ser conservador pero un poco menos
            print("Entorno detectado: Local")
            max_usage_percent = 80  # Usar máximo 80% de la memoria disponible en local
        
        # Calcular memoria máxima a usar
        max_memory_mb = available_mb * (max_usage_percent / 100)
        
        print(f"Memoria total del sistema: {total_mb:.0f} MB")
        print(f"Memoria disponible: {available_mb:.0f} MB")
        print(f"Memoria máxima a utilizar: {max_memory_mb:.0f} MB ({max_usage_percent}% de la disponible)")
        
        return max_memory_mb
    except Exception as e:
        print(f"Error al obtener información de memoria: {e}")
        # Valor conservador por defecto
        return 2000  # 2GB por defecto si no se puede detectar

def calculate_optimal_batch_size(mem_per_sample_mb, max_memory_mb, min_batch=10):
    """Calcula el tamaño óptimo de lote basado en la memoria disponible"""
    # Estimar cuánta memoria usa cada muestra (con margen de seguridad)
    optimal_batch = int(max_memory_mb / (mem_per_sample_mb * 1.5))
    return max(optimal_batch, min_batch)

def configure_optuna_for_memory_efficiency(max_memory_mb, model_name):
    """Configura parámetros para Optuna basados en la memoria disponible"""
    # Estimar número de trials y paralelismo basado en la memoria
    if max_memory_mb < 1000:  # Menos de 1GB disponible
        n_trials = 15  # Reducir número de trials
        n_jobs = 1     # Sin paralelismo
        use_sqlite = True  # Usar SQLite para ahorrar memoria
    elif max_memory_mb < 4000:  # Menos de 4GB
        n_trials = 25  # Número moderado de trials
        n_jobs = min(2, os.cpu_count() or 2)  # Paralelismo limitado
        use_sqlite = True  # Usar SQLite
    else:  # 4GB o más
        n_trials = 30  # Número normal de trials
        n_jobs = min(3, os.cpu_count() or 2)  # Mejor paralelismo
        use_sqlite = True  # Mantener SQLite para estabilidad
    
    # Definir configuración específica por modelo
    model_config = {
        'RandomForest': {
            'n_estimators_max': min(500, int(50 + max_memory_mb / 100)),  # Adaptar a memoria
            'max_depth_max': min(50, int(10 + max_memory_mb / 200))
        },
        'XGBoost': {
            'tree_method': 'hist',  # Método eficiente en memoria
            'max_depth_max': min(12, 3 + int(max_memory_mb / 1000)),
            'grow_policy': 'lossguide'  # Más eficiente en memoria
        },
        'LightGBM': {
            'max_bin': min(255, 63 + int(max_memory_mb / 100)),  # Adaptar precisión
            'num_leaves_max': min(150, 31 + int(max_memory_mb / 100))
        }
    }
    
    # Crear almacenamiento SQLite si es necesario
    storage = None
    if use_sqlite:
        # Crear directorio para almacenamiento Optuna si no existe
        os.makedirs(model_output_dir / 'optuna_storage', exist_ok=True)
        storage = f"sqlite:///{model_output_dir}/optuna_storage/optuna_{model_name}_{int(time.time())}.db"
    
    # Devolver toda la configuración optimizada
    return {
        'n_trials': n_trials,
        'n_jobs': n_jobs,
        'storage': storage,
        'model_config': model_config.get(model_name, {})
    }

def memory_efficient_objective_factory(model_name, X_train, y_train, max_memory_mb, model_config):
    """Crea una función objetivo para Optuna optimizada para memoria"""
    # Configurar espacio de búsqueda adaptado a la memoria disponible
    def objective(trial):
        # Limpiar memoria antes de cada trial
        gc.collect()
        
        # Configuración de parámetros específicos para cada modelo, adaptados a la memoria
        if model_name == 'RandomForest':
            n_estimators_max = model_config.get('n_estimators_max', 500)
            max_depth_max = model_config.get('max_depth_max', 50)
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, n_estimators_max),
                'max_depth': trial.suggest_int('max_depth', 5, max_depth_max),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'random_state': 42,
                # Parámetros para optimizar memoria
                'verbose': 0,
                'n_jobs': 1 if max_memory_mb < 2000 else 2  # Limitar paralelismo según memoria
            }
            model = RandomForestRegressor(**params)
        
        elif model_name == 'XGBoost':
            max_depth_max = model_config.get('max_depth_max', 12)
            tree_method = model_config.get('tree_method', 'hist')
            grow_policy = model_config.get('grow_policy', 'lossguide')
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, max_depth_max),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
                'random_state': 42,
                # Optimización de memoria
                'tree_method': tree_method,
                'grow_policy': grow_policy,
                'verbosity': 0,
                'nthread': 1 if max_memory_mb < 2000 else 2  # Adaptar según memoria
            }
            model = XGBRegressor(**params)
        
        elif model_name == 'LightGBM':
            num_leaves_max = model_config.get('num_leaves_max', 150)
            max_bin = model_config.get('max_bin', 63)
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, num_leaves_max),
                'max_depth': trial.suggest_int('max_depth', -1, 15),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
                'random_state': 42,
                # Parámetros para optimizar memoria
                'verbose': -1,
                'max_bin': max_bin,
                'num_threads': 1 if max_memory_mb < 2000 else 2
            }
            model = LGBMRegressor(**params)
        else:
            raise ValueError(f"Modelo no soportado: {model_name}")
        
        # Evaluación con validación cruzada adaptada a memoria disponible
        try:
            # Reducir el número de folds si hay poca memoria
            n_folds = 3 if max_memory_mb < 2000 else 5
            from sklearn.model_selection import KFold
            
            kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
            scores = []
            
            for train_idx, val_idx in kf.split(X_train):
                X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                
                # Entrenar modelo
                model.fit(X_fold_train, y_fold_train)
                
                # Evaluar y liberar memoria inmediatamente
                y_pred = model.predict(X_fold_val)
                rmse = np.sqrt(mean_squared_error(y_fold_val, y_pred))
                scores.append(rmse)
                
                # Limpieza agresiva de memoria
                del X_fold_train, X_fold_val, y_fold_train, y_fold_val, y_pred
                gc.collect()
            
            # Liberar modelo también
            del model
            gc.collect()
            
            return np.mean(scores)
        except Exception as e:
            print(f"Error en evaluación: {e}")
            return float('inf')
    
    return objective

def run_memory_efficient_optimization(model_name='XGBoost', X_train_scaled=None, y_train=None, X_test_scaled=None, y_test=None):
    """Ejecuta optimización con Optuna adaptada a la memoria disponible"""
    print(f"\n🧠 Iniciando optimización adaptativa de memoria para {model_name}...")
    
    # Verificar si se proporcionaron los datos
    if X_train_scaled is None or y_train is None or X_test_scaled is None or y_test is None:
        raise ValueError("Es necesario proporcionar los conjuntos de datos X_train_scaled, y_train, X_test_scaled y y_test")
    
    # 1. Detectar memoria disponible
    max_memory_mb = get_available_memory()
    
    # 2. Configurar Optuna según memoria disponible
    optuna_config = configure_optuna_for_memory_efficiency(max_memory_mb, model_name)
    
    # 3. Mostrar configuración
    print(f"\nConfiguración para {model_name}:")
    print(f"- Trials: {optuna_config['n_trials']}")
    print(f"- Paralelismo: {optuna_config['n_jobs']} jobs")
    print(f"- Almacenamiento: {'SQLite' if optuna_config['storage'] else 'En memoria'}")
    print(f"- Configuración específica: {optuna_config['model_config']}")
    
    # 4. Crear estudio Optuna
    from optuna.pruners import MedianPruner
    pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=5)
    
    study = optuna.create_study(
        direction="minimize",
        pruner=pruner,
        storage=optuna_config['storage'],
        study_name=f"{model_name}_memory_optimized",
        load_if_exists=True
    )
    
    # 5. Crear función objetivo eficiente en memoria
    objective = memory_efficient_objective_factory(
        model_name, 
        X_train_scaled, 
        y_train, 
        max_memory_mb,
        optuna_config['model_config']
    )
    
    # 6. Callback para liberar memoria periódicamente
    def gc_callback(study, trial):
        if trial.number % 3 == 0:  # Cada 3 trials
            gc.collect()
    
    # 7. Visualización inicial
    color_map = {
        'RandomForest': '#e6f3ff',  # Azul claro
        'XGBoost': '#fff0e6',      # Naranja claro
        'LightGBM': '#e6fff2'      # Verde claro
    }
    display(HTML(f'<div style="background-color:{color_map.get(model_name, "#f5f5f5")}; padding:10px; border-radius:5px;">' +
                f'<h3>🔍 Optimización adaptativa de memoria - {model_name}</h3>' +
                f'<div>Memoria máxima a utilizar: {max_memory_mb:.0f} MB</div>' +
                f'<div>Trials planeados: {optuna_config["n_trials"]}</div>' +
                f'</div>'))
    
    # 8. Iniciar optimización
    callback = OptimizationProgressCallback(
        optuna_config['n_trials'], f"{model_name} (RAM optimizada)"
    )
    
    study.optimize(
        objective, 
        n_trials=optuna_config['n_trials'],
        n_jobs=optuna_config['n_jobs'],
        gc_after_trial=True,
        callbacks=[callback, gc_callback]
    )
    
    # 9. Obtener mejores parámetros
    best_params = study.best_params
    best_value = study.best_value
    
    # 10. Visualizar resultados
    display(HTML(f'<div style="background-color:{color_map.get(model_name, "#f5f5f5")}; padding:10px; border-radius:5px;">' +
                f'<h3>✅ Optimización completada - {model_name}</h3>' +
                f'<div><b>Mejor RMSE:</b> {best_value:.4f}</div>' +
                f'<div><b>Mejores parámetros:</b> {str(best_params)}</div>' +
                f'</div>'))
    
    # 11. Entrenar modelo final con los mejores parámetros
    print("\nEntrenando modelo final con los mejores parámetros...")
    
    if model_name == 'RandomForest':
        best_model = RandomForestRegressor(**best_params, random_state=42)
    elif model_name == 'XGBoost':
        best_model = XGBRegressor(**best_params, random_state=42)
    elif model_name == 'LightGBM':
        best_model = LGBMRegressor(**best_params, random_state=42)
    
    # 12. Evaluar modelo final
    better_model, metrics = entrenar_y_evaluar_modelo(
        best_model, f'{model_name}_RAM_Opt', X_train_scaled, y_train, X_test_scaled, y_test
    )
    
    # 13. Guardar resultados
    resultados_base[f'{model_name}_RAM_Opt'] = metrics
    modelo_file = guardar_modelo(better_model, f'{model_name}_RAM_Opt')
    modelos_guardados[f'{model_name}_RAM_Opt'] = modelo_file
    
    return best_params, better_model, metrics

# Ejecutar solo esta celda para iniciar la optimización adaptativa de memoria para XGBoost
print("\n🔍 Ejecutando optimización adaptativa de memoria RAM para XGBoost...")
xgb_params, xgb_model, xgb_metrics = run_memory_efficient_optimization('XGBoost', X_train_scaled, y_train, X_test_scaled, y_test)